In [31]:
import torch
import torch.nn as nn
import torchvision
import torch.nn.functional as F
import torchvision.transforms as transforms

In [41]:
# device configuration
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# hyper parameters
num_epochs = 5
num_classes = 10
batch_size = 8
learning_rate = 0.0001

In [23]:
#mnist dataset
transforms = transforms.Compose([transforms.Resize((32, 32)),
                                 transforms.ToTensor()])

train_dataset = torchvision.datasets.MNIST(root="./data", download=True,
                                           transform=transforms,
                                           train=True)

test_dataset = torchvision.datasets.MNIST(root="./data", download=True,
                                          transform=transforms,
                                          train=False)

In [24]:
# data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=True)

## LeNet5

<p align="center">
  <img src="images/LeNet5.png">
</p>

In [33]:
class LeNet5(nn.Module):
    def __init__(self, n_classes):
        super(LeNet5, self).__init__()
        
        self.feature_extractor = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5, stride=1),
            nn.Tanh(),
            nn.AvgPool2d(kernel_size=2),
            nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5, stride=1),
            nn.Tanh(),
            nn.AvgPool2d(kernel_size=2),
            nn.Conv2d(in_channels=16, out_channels=120, kernel_size=5, stride=1),
            nn.Tanh()
        )
        
        self.classifier = nn.Sequential(
            nn.Linear(in_features=120, out_features=84),
            nn.Tanh(),
            nn.Linear(in_features=84, out_features=n_classes)
        )
        
    def forward(self, x):
        x = self.feature_extractor(x)
        x = torch.flatten(x, 1)
        logits = self.classifier(x)
        probs = F.softmax(logits, dim=1)
        
        return logits, probs

In [42]:
# get a model instance
model = LeNet5(n_classes=num_classes).to(device)

# loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [43]:
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        
        # forward pass
        outputs, _ = model(images)
        train_loss = criterion(outputs, labels)
        
        # backward and optimization
        optimizer.zero_grad()
        train_loss.backward()
        optimizer.step()
        
    # evaluate the model on the test set    
    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)
            outputs, _ = model(images)
            test_loss = criterion(outputs, labels)
            
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        
    print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss.item()}, Test Loss: {test_loss.item()}")
    print(f"Test accuracy: {100 * correct / total}")
        

Epoch [1/5], Train Loss: 0.18992525339126587, Test Loss: 0.20220623910427094
Test accuracy: 93.2
Epoch [2/5], Train Loss: 0.0374927744269371, Test Loss: 0.029778197407722473
Test accuracy: 95.74
Epoch [3/5], Train Loss: 0.09928139299154282, Test Loss: 0.07281767576932907
Test accuracy: 96.79
Epoch [4/5], Train Loss: 0.4637470245361328, Test Loss: 0.12256816029548645
Test accuracy: 97.33
Epoch [5/5], Train Loss: 0.020538879558444023, Test Loss: 0.027356769889593124
Test accuracy: 97.85


In [44]:
# save the model checkpoint
torch.save(model.state_dict(), 'LeNet5.pt')